# Install Libraries
! pip install swarmauri[full]==0.4.1 python-dotenv


In [21]:
from dotenv import load_dotenv
import os
import gradio as gr
from swarmauri.standard.llms.concrete.GroqModel import GroqModel
from swarmauri.standard.messages.concrete.SystemMessage import SystemMessage
from swarmauri.standard.agents.concrete.SimpleConversationAgent import SimpleConversationAgent
from swarmauri.standard.conversations.concrete.MaxSystemContextConversation import MaxSystemContextConversation

# Load environment variables from .env file
load_dotenv()

# Fetch the API key from environment variables
API_KEY = os.getenv("API_KEY")

# Initialize the GroqModel with the API key to access allowed models
llm = GroqModel(api_key=API_KEY)

# Get the available models from the llm instance
allowed_models = llm.allowed_models

# Initialize a MaxSystemContextConversation instance
conversation = MaxSystemContextConversation()

# Define a function to dynamically change the model based on dropdown input
def load_model(selected_model):
    return GroqModel(api_key=API_KEY, name=selected_model)

# Define the function to interact with the agent
def converse(input_text, history, system_context, model_name):
    print(f"system_context: {system_context}")
    print(f"Selected model: {model_name}")
    
    # Initialize the model dynamically based on user selection
    llm = load_model(model_name)
    
    # Initialize the agent with the new model
    agent = SimpleConversationAgent(llm=llm, conversation=conversation)
    
    agent.conversation.system_context = SystemMessage(content=system_context)
    
    # Ensure input_text is a string
    input_text = str(input_text)
    print(conversation.history)
    # Execute the input command with the agent
    result = agent.exec(input_text)
    print(result, type(result))
    
    # Return the result as a string
    return str(result)

# Set up the Gradio ChatInterface with a dropdown for model selection
demo = gr.ChatInterface(
    fn=converse,
    additional_inputs=[
        gr.Textbox(label="System Context"),
        gr.Dropdown(label="Model Name", choices=allowed_models, value=allowed_models[0])  # Dropdown with allowed models
    ],
    title="A system context conversation",
    description="Interact with the agent using a selected model and system context."
)

if __name__ == "__main__":
    demo.launch()


Running on local URL:  http://127.0.0.1:7917
IMPORTANT: You are using gradio version 4.26.0, however version 4.29.0 is available, please upgrade.
--------

To create a public link, set `share=True` in `launch()`.
